In [8]:
"""
Alvaro Rafael Gomez

task 006 - part 1 - testing qiskit "quantum_info.partial_trace" to calculate output density matrix

Code based on "Resource 1":  
https://github.com/Advanced-Research-Centre/QKSA/blob/main/legacy_vers/v12/sandbox/aaqpt.py

Obtained results are consistent with "Resource 1".
"""

from qiskit import *
import numpy as np

process_qubits = 1
qubits = 2*process_qubits
hsdim = 2**qubits

'''
The "black-box" / "unknown" quantum process
'''
def QProcess(qcirc, qreg):
    qcirc.barrier()
    qcirc.h(qreg[0])
    qcirc.barrier()
    return
'''
Make ancilla entangled state (Bell00)
'''
def EntangleAncilla(qcirc):
    qcirc.barrier()
    qcirc.h(1)
    qcirc.cx(1,0)
    qcirc.barrier()
    return

'''
Actual input and output density matrices and normalized choi of process subsystem
'''
def Actual_Input_Output():
    psi_inp = QuantumCircuit(process_qubits, process_qubits)
    psi_inp.i(0)
    act_rho_inp = quantum_info.DensityMatrix.from_instruction(psi_inp).data
    #Appending black box process
    QProcess(psi_inp, [0])
    act_rho_out = quantum_info.DensityMatrix.from_instruction(psi_inp).data
    rho_choi = quantum_info.Choi(psi_inp).data / 2**psi_inp.num_qubits
    return act_rho_inp, act_rho_out, rho_choi


qcirc = QuantumCircuit(qubits, qubits, name = "qcirc")
EntangleAncilla(qcirc)
QProcess(qcirc, [0])
print("\nCircuit to test")                                                                              
print(qcirc)
act_rho_inp, act_rho_out, rho_choi = Actual_Input_Output() 

#Estimated output density matrix
est_output_dm = quantum_info.partial_trace(np.matmul(np.kron(np.transpose(act_rho_inp), np.eye(2**process_qubits) ), rho_choi),[1])

print("\nOutput Density Matrix")
print(est_output_dm)



Circuit to test
      ░      ┌───┐ ░  ░ ┌───┐ ░ 
q_0: ─░──────┤ X ├─░──░─┤ H ├─░─
      ░ ┌───┐└─┬─┘ ░  ░ └───┘ ░ 
q_1: ─░─┤ H ├──■───░──░───────░─
      ░ └───┘      ░  ░       ░ 
c: 2/═══════════════════════════
                                

Output Density Matrix
DensityMatrix([[0.25+0.j, 0.25+0.j],
               [0.25+0.j, 0.25+0.j]],
              dims=(2,))
